In [3]:
import threading

# 创建全局ThreadLocal对象
local_school = threading.local()


def process_thread(name):
    # 绑定ThreadLocal的student
    local_school.student = name

    # 获取当前线程关联的student
    std = local_school.student
    print(f"你好,{std}(in {threading.current_thread().name})")


t1 = threading.Thread(target=process_thread, args=("Alice",), name="线程-A")
t2 = threading.Thread(target=process_thread, args=("Bob",), name="线程-B")

t1.start()
t2.start()
t1.join()
t2.join()

你好,Alice(in 线程-A)
你好,Bob(in 线程-B)


#### 全局变量local_school就是一个ThreadLocal对象,每个线程都可以读写student属性，但互不影响
#### 将local_school理解全局变量，但每个属性如local_school.student都是线程的局部变量，可以任意读写而互不干扰，也不用管理锁的问题，ThreadLocal内部会处理。
#### ThreadLocal最常用的地方就是为每个线程绑定一个数据库连接，HTTP请求，用户身份信息

## 来自 https://selfboot.cn/2016/08/22/threadlocal_overview/ 的教程

### 全局变量 & 局部变量

In [39]:
import threading

global_num = 0


def call():
    global global_num
    for _ in range(100000):
        global_num += _


# 得到10个线程，运行他们并等待结束
threads = []
for _ in range(10):
    threads.append(threading.Thread(target=call))
    threads[_].start()

for _ in range(10):
    threads[_].join()

print(global_num)

45742052080


In [31]:
from functools import reduce

L = [i for i in range(100000)]
reduce(lambda _1, _2: _1 + _2, L)

4999950000

In [34]:
# 对同一代码加锁 加锁
import threading

global_num = 0
p = threading.Lock()

def call():
    global global_num
    
    for _ in range(100000):
        global_num += _

def call_lock():
    p.acquire()
    try:
        call()
    finally:
        p.release()

# 得到10个线程，运行他们并等待结束
threads = []
for _ in range(10):
    threads.append(threading.Thread(target=call_lock))
    threads[_].start()

for _ in range(10):
    threads[_].join()

print(global_num)

49999500000


In [37]:
# 局部变量演示
import threading

def func():
    局部变量 = 0
    for _ in range(100000):
        局部变量 += _
    print(局部变量,threading.current_thread().name)


threads = []
for _ in range(10):
    threads.append(threading.Thread(target=func))
    threads[_].start()
    threads[_].join()

4999950000 Thread-174
4999950000 Thread-175
4999950000 Thread-176
4999950000 Thread-177
4999950000 Thread-178
4999950000 Thread-179
4999950000 Thread-180
4999950000 Thread-181
4999950000 Thread-182
4999950000 Thread-183


#### ThreadLocal 真正做到了线程隔离，并且不需要自己获取线程ID